In [4]:

import os
from langchain_groq import ChatGroq


os.environ["GROQ_API_KEY"] = "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" 

if not os.environ["GROQ_API_KEY"].startswith("gsk_"):
    raise ValueError("❌ STOP! You forgot to paste your API Key in Cell 1.")

try:
    llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
    print("✅ Guardrail Agent Initialized.")
except Exception as e:
    print(f"❌ Error: {e}")


✅ Guardrail Agent Initialized.


In [5]:
def safety_check(report_text):
    """
    Acts as a compliance officer. 
    Returns (True, report) if safe.
    Returns (False, warning) if unsafe.
    """
    print("🛡️ GUARDRAILS: Scanning output for financial compliance...")
    
    system_prompt = """
    You are a Financial Compliance Officer. Review the following investment report.
    
    FAIL CRITERIA:
    1. Promises "guaranteed" returns.
    2. Tells the user to "invest their life savings."
    3. Uses definitive language like "You must buy" instead of "Recommendation."
    
    If it passes, reply ONLY with: SAFE
    If it fails, reply with: UNSAFE: <reason>
    """
    
    response = llm.invoke(f"{system_prompt}\n\nREPORT:\n{report_text}")
    result = response.content.strip()
    
    if result.startswith("SAFE"):
        return True, report_text
    else:
        # Return a sanitized version
        warning = f"⚠️ CONTENT BLOCKED BY GUARDRAILS.\nReason: {result}\n\nPlease consult a licensed financial advisor."
        return False, warning

In [6]:
print("--- TEST 1: Safe Content ---")
safe_report = "The stock shows strong technical support. We recommend a BUY with a stop loss at 2400."
is_safe, msg = safety_check(safe_report)
print(f"Result: {msg}\n")

print("--- TEST 2: Unsafe Content (The Hallucination) ---")
unsafe_report = "I guarantee you will become a millionaire if you buy this stock! Put your life savings in now! 100% returns guaranteed!"
is_safe, msg = safety_check(unsafe_report)
print(f"Result: {msg}")

--- TEST 1: Safe Content ---
🛡️ GUARDRAILS: Scanning output for financial compliance...
Result: The stock shows strong technical support. We recommend a BUY with a stop loss at 2400.

--- TEST 2: Unsafe Content (The Hallucination) ---
🛡️ GUARDRAILS: Scanning output for financial compliance...
Result: ⚠️ CONTENT BLOCKED BY GUARDRAILS.
Reason: UNSAFE: The report promises "guaranteed" returns, tells the user to "invest their life savings", and uses definitive language implying a certain outcome.

Please consult a licensed financial advisor.
